In [1]:
from sail.data import DataFrameGroup
from sail.algo import Fdlr
from sail.core import spawnvms
import optuna

In [2]:
vms = spawnvms(3)
vmagg = vms[-1]
vms = vms[:-1]

In [3]:
workplace = "/home/jjj/playground/tmp"

In [4]:
dfg = DataFrameGroup(vms, workplace)
dfg.import_data([0,0])

In [5]:
dfg.shape

[(3, 6), (2, 6)]

In [6]:
dfg.col_label

[Index(['Region', 'Temp(F)', 'Rainfall(mm)', 'Humidity(%)', 'Apples(ton)',
        'Oranges(ton)'],
       dtype='object'),
 Index(['Region', 'Temp(F)', 'Rainfall(mm)', 'Humidity(%)', 'Apples(ton)',
        'Oranges(ton)'],
       dtype='object')]

In [7]:
dfg.sample(vms[0], dfg.df[0])

,Region,Temp(F),Rainfall(mm),Humidity(%),Apples(ton),Oranges(ton)
0,Kanto,73,67,43,56,70
1,Johto,91,88,64,81,101
2,Hoenn,87,124,58,119,133


In [8]:
#mu = dfg.mean('price')

In [9]:
#mu

In [10]:
#sigma = dfg.standard_deviation('price', mu)

In [11]:
#sigma

In [12]:
#sk = dfg.skew('price', mu, sigma)

In [13]:
#sk

In [14]:
#dfg.log_transform(['price'], ['log_price'])

In [15]:
data = {}
X = dfg.get_col([['Temp(F)', 'Rainfall(mm)', 'Humidity(%)']]*len(vms), dfg.df)
y = dfg.get_col([['Apples(ton)', 'Oranges(ton)']]*len(vms), dfg.df)
data['X_train'] = dfg.to_numpy(X)
data['y_train'] = dfg.to_numpy(y)
#data['X_train'], data['X_test'], data['y_train'], data['y_test'] = dfg.train_test_split(['log_price'], ['sqft_living15'], 0.2)

In [16]:
model = Fdlr(vms, vmagg, data, workplace)

In [17]:
def objective(trail, model):
    n_lr = trail.suggest_float('n_lr', 1e-5, 1e-4, step = 3e-4)
    n_epochs = trail.suggest_int('n_epochs', 50, 200, step = 50)
    model.initmodel(3,2)
    model.initsubmodels(n_lr)
    model.train(n_epochs)
    return model.score('neg_mean_absolute_error', 5)

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(lambda trial: objective(trial, model), n_trials=3)

[I 2021-04-13 15:58:01,030] A new study created in memory with name: no-name-a6cca606-221b-49d5-92f8-d40e51309d86


In [16]:
model.initmodel(1,1)
model.initsubmodels()

In [17]:
model.train(100)

processing round: 1
get_grad: push data on vm0
get_grad: run job on vm0
get_grad: pull data on vm0
get_grad: push data on vm1
get_grad: run job on vm1
get_grad: pull data on vm1
agg: push data on vm
agg: run job on vm
agg: pull data on vm
update_grad: push data on vm0
update_grad: run job on vm0
update_grad: pull data on vm0
update_grad: push data on vm1
update_grad: run job on vm1
update_grad: pull data on vm1
processing round: 2
get_grad: push data on vm0
get_grad: run job on vm0
get_grad: pull data on vm0
get_grad: push data on vm1
get_grad: run job on vm1
get_grad: pull data on vm1
agg: push data on vm
agg: run job on vm
agg: pull data on vm
update_grad: push data on vm0
update_grad: run job on vm0
update_grad: pull data on vm0
update_grad: push data on vm1
update_grad: run job on vm1
update_grad: pull data on vm1
processing round: 3
get_grad: push data on vm0
get_grad: run job on vm0
get_grad: pull data on vm0
get_grad: push data on vm1
get_grad: run job on vm1
get_grad: pull data

In [18]:
model.mae()

[1379.724577293396, 1373.2931090164184]